In [1]:
import torch
import torch_geometric
import os 
from dataset import GraphDataset
from model.hand_made import GNNPolicy
from readdataUC import UC
import pandas as pd
from scipy.sparse import coo_matrix
from torch.optim.lr_scheduler import CosineAnnealingLR
import time

In [2]:
# DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#set folder
model_save_path = './model_save/1bin_new_part_loss/'
log_save_path = "./model_save/1bin_new_part_loss/"
if not os.path.isdir(model_save_path):
    os.mkdir(model_save_path)
if not os.path.isdir(log_save_path):
    os.mkdir(log_save_path)
log_file = open(f'{log_save_path}train.log', 'ab')

In [3]:
LEARNING_RATE = 5e-4
NB_EPOCHS =9999
BATCH_SIZE = 1
NUM_WORKERS = 2
ETA_MIN=1e-6

In [4]:
# train_data = GraphDataset('dataset_20_0_1_w/1bin/train/')
# train_iter = torch_geometric.loader.DataLoader(
#     train_data, batch_size = BATCH_SIZE, shuffle = True,
#     num_workers = NUM_WORKERS, pin_memory=True)
# valid_data = GraphDataset('dataset_20_0_1_w/1bin/valid/')
# valid_iter = torch_geometric.loader.DataLoader(
#     valid_data, batch_size = BATCH_SIZE, shuffle = False,
#     num_workers = NUM_WORKERS)

In [5]:
data = GraphDataset('dataset_20_0_1_w/1bin/train/')
split_idx = int(len(data) * 0.8)
train_data = data[:split_idx]
valid_data = data[split_idx:]
train_iter = torch_geometric.loader.DataLoader(
    train_data, batch_size = BATCH_SIZE, shuffle = True,
    num_workers = NUM_WORKERS, pin_memory=True)
valid_iter = torch_geometric.loader.DataLoader(
    valid_data, batch_size = BATCH_SIZE, shuffle = False,
    num_workers = NUM_WORKERS)

In [6]:
PredictModel = GNNPolicy().to(DEVICE)
if os.path.exists(model_save_path+'model_best.pth'):
    state_dict = torch.load(model_save_path+'model_best.pth')
    PredictModel.load_state_dict(state_dict)
    print("载入模型成功")


载入模型成功


In [7]:

def get_edge_index(N,T):

    source_node=[]
    target_node=[]
    for i in range(N):
        for t in range(T):
            j=i*T+t
            source_node.append(j)
            target_node.append(j)
            source_node.append(j+N*T)
            target_node.append(j)
            source_node.append(j+2*N*T)
            target_node.append(j)
            for ii in range(N):
                if ii != i:
                    source_node.append(j)
                    target_node.append(ii*T+t)
            for tt in range(T):
                if tt != t:
                    source_node.append(j)
                    target_node.append(i*T+tt)
    edge_index=torch.tensor([source_node,target_node], dtype=torch.long).to(DEVICE)
    return edge_index


In [8]:
def train(predict, data_loader, optimizer=None):
    """
    This function will process a whole epoch of training or validation, depending on whether an optimizer is provided.
    """
    loss_fn = torch.nn.BCELoss()

    if optimizer:
        predict.train()
    else:
        predict.eval()
    mean_loss = 0
    n_samples_processed = 0
    with torch.set_grad_enabled(optimizer is not None):
        for step, batch in enumerate(data_loader):
            batch = batch.to(DEVICE)
            weighted_sol = batch.weighted_sol
            #predict the binary distribution, BD
            # print(batch.variable_features.shape)
            N=int(batch.variable_features.shape[0]/24/3)
            T=24
            edge_index=get_edge_index(N,T)
            BD = predict(
                batch.constraint_features,
                batch.edge_index,
                batch.edge_attr,
                batch.variable_features,
                edge_index
            )
            
            # compute loss
            
            presol = BD[batch.b_vars==1]
            presol = presol.sigmoid()
            # print(batch.variable_features.shape)

            loss = loss_fn(presol, weighted_sol)

            if optimizer is not None:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            mean_loss += loss.item()
            n_samples_processed += 1
    mean_loss /= n_samples_processed

    return mean_loss


In [9]:
optimizer = torch.optim.Adam(PredictModel.parameters(), lr=LEARNING_RATE)
scheduler = CosineAnnealingLR(optimizer, T_max=NB_EPOCHS, eta_min=ETA_MIN)
best_val_loss = 0.2

In [10]:
for epoch in range(NB_EPOCHS):
    begin=time.time()
    train_loss = train(PredictModel, train_iter, optimizer)
    print(f"Epoch {epoch} Train loss: {train_loss:0.7f}",end='\t')
    valid_loss = train(PredictModel, valid_iter, None)
    print(f"Epoch {epoch} Valid loss: {valid_loss:0.7f}")
    if valid_loss<best_val_loss:
        best_val_loss = valid_loss
        print(best_val_loss)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best.pth')
    torch.save(PredictModel.state_dict(), model_save_path+'model_last.pth')
    date_str = time.strftime('%Y-%m-%d  %H:%M:%S', time.localtime())
    st = f'@epoch{epoch}   Train loss:{train_loss}   Valid loss:{valid_loss}    TIME:{time.time()-begin}    DONE_TIME:{date_str}\n'
    log_file.write(st.encode())
    log_file.flush()
    scheduler.step()
print('done')

Epoch 0 Train loss: 0.2276600	Epoch 0 Valid loss: 0.2292726
Epoch 1 Train loss: 0.2085280	Epoch 1 Valid loss: 0.2034287
Epoch 2 Train loss: 0.2026900	Epoch 2 Valid loss: 0.2009516
Epoch 3 Train loss: 0.1992294	Epoch 3 Valid loss: 0.2270362
Epoch 4 Train loss: 0.1956956	Epoch 4 Valid loss: 0.1989207
0.19892065344791643
Epoch 5 Train loss: 0.1940276	Epoch 5 Valid loss: 0.1983993
0.19839927992333395
Epoch 6 Train loss: 0.1921857	Epoch 6 Valid loss: 0.1924358
0.19243579861739124
Epoch 7 Train loss: 0.1909797	Epoch 7 Valid loss: 0.1944941
Epoch 8 Train loss: 0.1890817	Epoch 8 Valid loss: 0.2004128
Epoch 9 Train loss: 0.1880967	Epoch 9 Valid loss: 0.1975743
Epoch 10 Train loss: 0.1875701	Epoch 10 Valid loss: 0.1954195
Epoch 11 Train loss: 0.1867149	Epoch 11 Valid loss: 0.1905332
0.19053321278520993
Epoch 12 Train loss: 0.1850428	Epoch 12 Valid loss: 0.1965612
Epoch 13 Train loss: 0.1846385	Epoch 13 Valid loss: 0.2014644
Epoch 14 Train loss: 0.1839416	Epoch 14 Valid loss: 0.1889490
0.18894902

KeyboardInterrupt: 